In [1]:
import pandas as pd
import numpy as np
from collections import OrderedDict

In [2]:
models = {'cifar10': 'CIFAR-10',
          'cifar100': 'CIFAR-100',
          'imagenet200': 'Imagenet-200'}
methods = {'simple': 'No Detector',
           'class_mahalanobis': 'Class Mahananobis (OUR)',
           'simple2': 'Simple Mahalanobis (OUR)',
           'she': 'SHE \cite{she23iclr}', 
           'ash': 'ASH \cite{djurisic2023extremely}',
           'rankfeat': 'RankFeat \cite{song2022rankfeat}',
           'dice': 'DICE \cite{sun2021dice}',
           'knn': 'KNN \cite{sun2022knnood}',
           'vim': 'VIM \cite{haoqi2022vim}',
           #'kl_matching': 'KLM \cite{species22icml}',
           'mls': 'MLS \cite{species22icml}',
           'react': 'ReAct \cite{react21nips}',
           'gradnorm': 'GradNorm \cite{huang2021importance}',
           'ebo': 'EBO \cite{energyood20nips}',
           'rmds': 'RMDS \cite{rmd21arxiv}',
           'mds_ensemble': 'MDSEns \cite{mahananobis18nips}',
           'mds': 'MDS \cite{mahananobis18nips}',
           'odin': 'ODIN \cite{odin18iclr}',
           'scale': 'TempScale \cite{guo2017calibration}',
           'msp': 'MSP \cite{msp17iclr}'}
paths = {'cifar10': './results/cifar10_resnet18_32x32_base_e100_lr0.1_default/',
         'cifar100': './results/cifar100_resnet18_32x32_base_e100_lr0.1_default/',
         'imagenet200': './results/imagenet200_resnet18_224x224_base_e90_lr0.1_default/'}
atts = ['fgsm', 'pgd', 'igm', 'iter', 'adv']

In [3]:
data = {}
for method, method_name in methods.items():
    data[method] = {}
    for model, model_name in models.items():
        csv = pd.read_csv(paths[model]+f'ood/{method}.csv', index_col=0)
        dtime, stime = csv.at['adv', 'TIME'], csv.at['setup_time', 'TIME']
        resat = csv['AUROC'].loc[['fgsm', 'pgd', 'igm', 'iter', 'adv']].values
        d = csv[['FPR@95','AUROC','AUPR_IN','AUPR_OUT']].loc[['fgsm', 'pgd', 'igm', 'iter', 'adv']].values
        data[method][model] = {'Exec. (ms)': dtime, 'Setup (s)': stime, 'AUROC': resat, 'd': d}

In [4]:
def relatex(string, rn=lambda x: x):
    a, b = string.split(' ± ')
    return rn(a)+'\\footnotesize{\\textpm' + rn(b) + '}'

def dmil(string):
    v = float(string) / 1000
    return f'{v:.2f}'

In [38]:
dtimes = {}
for model, model_name in models.items():
    for param in ['Exec. (ms)', 'Setup (s)']:
        tup = (model_name, param)
        dtimes[tup] = {}
        for method, method_name in methods.items():
            dtimes[tup][method_name] = relatex(data[method][model][param], dmil if param=='Exec. (ms)' else lambda x:x)

dtimesdf = pd.DataFrame.from_dict(dtimes)
print(dtimesdf.to_latex())

\begin{tabular}{lllllll}
\toprule
 & \multicolumn{2}{r}{CIFAR-10} & \multicolumn{2}{r}{CIFAR-100} & \multicolumn{2}{r}{Imagenet-200} \\
 & Exec. (ms) & Setup (s) & Exec. (ms) & Setup (s) & Exec. (ms) & Setup (s) \\
\midrule
No Detector & 22.51\footnotesize{\textpm0.28} & 3.85\footnotesize{\textpm0.06} & 23.42\footnotesize{\textpm0.15} & 3.95\footnotesize{\textpm0.15} & 108.63\footnotesize{\textpm2.14} & 271.60\footnotesize{\textpm2.76} \\
Class Mahananobis (OUR) & 24.59\footnotesize{\textpm0.15} & 4.85\footnotesize{\textpm0.06} & 25.01\footnotesize{\textpm0.04} & 5.10\footnotesize{\textpm0.11} & 119.71\footnotesize{\textpm2.95} & 276.85\footnotesize{\textpm1.69} \\
Simple Mahalanobis (OUR) & 23.66\footnotesize{\textpm0.18} & 4.59\footnotesize{\textpm0.11} & 24.14\footnotesize{\textpm0.15} & 4.58\footnotesize{\textpm0.09} & 111.08\footnotesize{\textpm1.58} & 272.44\footnotesize{\textpm0.33} \\
SHE \cite{she23iclr} & 23.32\footnotesize{\textpm0.31} & 4.68\footnotesize{\textpm0.08} & 23.1

In [6]:
data['simple']

{'cifar10': {'Exec. (ms)': '22514.29 ± 278.28',
  'Setup (s)': '3.85 ± 0.06',
  'AUROC': array(['79.91 ± 4.64', '49.97 ± 27.71', '60.20 ± 26.49', '45.19 ± 24.92',
         '58.82 ± 19.58'], dtype=object),
  'd': array([['79.40 ± 9.53', '79.91 ± 4.64', '82.85 ± 3.91', '68.98 ± 6.10'],
         ['97.46 ± 2.02', '49.97 ± 27.71', '55.64 ± 13.37',
          '55.44 ± 18.07'],
         ['90.84 ± 6.67', '60.20 ± 26.49', '68.44 ± 16.88',
          '58.46 ± 19.51'],
         ['99.49 ± 0.36', '45.19 ± 24.92', '49.50 ± 10.17',
          '53.49 ± 16.36'],
         ['91.80 ± 4.25', '58.82 ± 19.58', '64.11 ± 10.27',
          '59.09 ± 13.16']], dtype=object)},
 'cifar100': {'Exec. (ms)': '23420.51 ± 151.94',
  'Setup (s)': '3.95 ± 0.15',
  'AUROC': array(['90.72 ± 1.53', '32.11 ± 9.92', '59.67 ± 12.30', '36.28 ± 11.85',
         '54.70 ± 8.81'], dtype=object),
  'd': array([['31.34 ± 3.98', '90.72 ± 1.53', '94.69 ± 0.93', '82.47 ± 3.31'],
         ['99.96 ± 0.03', '32.11 ± 9.92', '47.06 ± 3.22', '36.

In [47]:
dadvs = {}
for model, model_name in models.items():
    tup = (model_name, 'IGMM')
    tupm = (model_name, 'Comb.')
    dadvs[tup] = {}
    dadvs[tupm] = {}
    for method, method_name in methods.items():
        rocs = data[method][model]['AUROC']
        dadvs[tup][method_name] = relatex(rocs[2])
        dadvs[tupm][method_name] = relatex(rocs[-1])

dadvsdf = pd.DataFrame.from_dict(dadvs)
print(dadvsdf.drop('No Detector', axis=0).to_latex())

\begin{tabular}{lllllll}
\toprule
 & \multicolumn{2}{r}{CIFAR-10} & \multicolumn{2}{r}{CIFAR-100} & \multicolumn{2}{r}{Imagenet-200} \\
 & IGMM & Comb. & IGMM & Comb. & IGMM & Comb. \\
\midrule
Class Mahananobis (OUR) & 84.77\footnotesize{\textpm3.55} & 88.53\footnotesize{\textpm0.28} & 81.00\footnotesize{\textpm2.58} & 81.19\footnotesize{\textpm5.75} & 88.60\footnotesize{\textpm3.78} & 90.44\footnotesize{\textpm2.64} \\
Simple Mahalanobis (OUR) & 84.94\footnotesize{\textpm3.03} & 90.29\footnotesize{\textpm3.56} & 71.57\footnotesize{\textpm6.25} & 73.67\footnotesize{\textpm7.55} & 73.90\footnotesize{\textpm13.43} & 71.65\footnotesize{\textpm7.41} \\
SHE \cite{she23iclr} & 52.51\footnotesize{\textpm24.94} & 50.26\footnotesize{\textpm19.24} & 54.39\footnotesize{\textpm11.48} & 51.25\footnotesize{\textpm8.01} & 44.14\footnotesize{\textpm22.37} & 50.24\footnotesize{\textpm15.25} \\
ASH \cite{djurisic2023extremely} & 49.07\footnotesize{\textpm23.30} & 48.62\footnotesize{\textpm16.22} & 57.7

In [ ]:
d = {
    ('Sales', 'Total'): [100, 150, 200, 250],
    ('Sales', 'Online'): [50, 70, 100, 150],
    ('Customers', 'Total'): [20, 25, 30, 35],
    ('Customers', 'Online'): [10, 15, 20, 25]
}

# Step 2: Create a MultiIndex for the rows
row_index = pd.MultiIndex.from_tuples([
    ('Store 1', '2020'),
    ('Store 1', '2021'),
    ('Store 2', '2020'),
    ('Store 2', '2021')
], names=['Store', 'Year'])

# Step 3: Create a MultiIndex for the columns
column_index = pd.MultiIndex.from_tuples([
    ('Sales', 'Total'),
    ('Sales', 'Online'),
    ('Customers', 'Total'),
    ('Customers', 'Online')
], names=['Metric', 'Type'])

# Step 4: Create the DataFrame
df = pd.DataFrame(np.arange(16).reshape(4, 4), index=row_index, columns=column_index)
df

In [29]:
deta = {}
atcs = ['FGSM', 'PGD', 'IGM', 'Iter.', 'Mean']
mets = ['FPR@95','AUROC','AUPRC\\textsubscript{IN}','AUPRC\\textsubscript{OUT}']
clidx = []
for model, model_name in models.items():
    for method, method_name in zip(['simple2', 'class_mahalanobis'], ['Sim. M.', 'Cla. M.']):
        tup = (model_name, method_name)
        deta[tup] = []
        clidx.append(tup)
        d = data[method][model]['d']
        for i in range(len(atcs)):
            for j in range(len(mets)):
                deta[tup].append(relatex(d[i, j]))

rwidx = [(a, m) for a in atcs for m in mets]
row_index = pd.MultiIndex.from_tuples(rwidx)
column_index = pd.MultiIndex.from_tuples(clidx)
detadf = pd.DataFrame(deta, index=row_index, columns=column_index)
print(detadf.to_latex())

\begin{tabular}{llllllll}
\toprule
 &  & \multicolumn{2}{r}{CIFAR-10} & \multicolumn{2}{r}{CIFAR-100} & \multicolumn{2}{r}{Imagenet-200} \\
 &  & Sim. M. & Cla. M. & Sim. M. & Cla. M. & Sim. M. & Cla. M. \\
\midrule
\multirow[t]{4}{*}{FGSM} & FPR@95 & 14.17\footnotesize{\textpm7.36} & 15.91\footnotesize{\textpm2.52} & 87.40\footnotesize{\textpm6.83} & 46.03\footnotesize{\textpm10.37} & 91.80\footnotesize{\textpm4.45} & 37.07\footnotesize{\textpm8.25} \\
 & AUROC & 96.96\footnotesize{\textpm1.59} & 95.28\footnotesize{\textpm0.72} & 56.20\footnotesize{\textpm13.32} & 83.38\footnotesize{\textpm6.26} & 51.57\footnotesize{\textpm7.42} & 90.20\footnotesize{\textpm2.48} \\
 & AUPRC\textsubscript{IN} & 98.12\footnotesize{\textpm0.98} & 97.43\footnotesize{\textpm0.37} & 68.73\footnotesize{\textpm9.87} & 90.38\footnotesize{\textpm3.74} & 65.44\footnotesize{\textpm6.15} & 94.19\footnotesize{\textpm1.62} \\
 & AUPRC\textsubscript{OUT} & 95.22\footnotesize{\textpm2.43} & 90.07\footnotesize{\textpm2

In [34]:
data

{'simple': {'cifar10': {'Exec. (ms)': '22514.29 ± 278.28',
   'Setup (s)': '3.85 ± 0.06',
   'AUROC': array(['79.91 ± 4.64', '49.97 ± 27.71', '60.20 ± 26.49', '45.19 ± 24.92',
          '58.82 ± 19.58'], dtype=object)},
  'cifar100': {'Exec. (ms)': '23420.51 ± 151.94',
   'Setup (s)': '3.95 ± 0.15',
   'AUROC': array(['90.72 ± 1.53', '32.11 ± 9.92', '59.67 ± 12.30', '36.28 ± 11.85',
          '54.70 ± 8.81'], dtype=object)},
  'imagenet200': {'Exec. (ms)': '108633.42 ± 2138.19',
   'Setup (s)': '271.60 ± 2.76',
   'AUROC': array(['93.71 ± 0.85', '42.41 ± 24.53', '53.04 ± 25.68', '41.73 ± 24.06',
          '57.72 ± 18.44'], dtype=object)}},
 'class_mahalanobis': {'cifar10': {'Exec. (ms)': '24590.68 ± 148.52',
   'Setup (s)': '4.85 ± 0.06',
   'AUROC': array(['95.28 ± 0.72', '87.21 ± 2.04', '84.77 ± 3.55', '86.88 ± 0.39',
          '88.53 ± 0.28'], dtype=object)},
  'cifar100': {'Exec. (ms)': '25010.53 ± 39.49',
   'Setup (s)': '5.10 ± 0.11',
   'AUROC': array(['83.38 ± 6.26', '81.58 ± 8

In [34]:
model = 'cifar10'
method = 'simple2'
csv = pd.read_csv(paths[model]+f'ood/{method}.csv', index_col=0)
csv#.at['adv', 'TIME'], csv.at['setup_time', 'TIME']
csv['AUROC'].loc[['fgsm', 'pgd', 'igm', 'iter', 'adv']].values

array(['96.96 ± 1.59', '88.18 ± 6.94', '84.94 ± 3.03', '91.08 ± 3.60',
       '90.29 ± 3.56'], dtype=object)

In [8]:
# Nested dictionary
d = {
    ('Store 1', '2020'): {'Sales': {'A': 100, 'B': 200}, 'Customers': 10},
    ('Store 1', '2021'): {'Sales': {'A': 150, 'B': 200}, 'Customers': 15},
    ('Store 2', '2020'): {'Sales': {'A': 200, 'B': 200}, 'Customers': 20},
    ('Store 2', '2021'): {'Sales': {'A': 250, 'B': 200}, 'Customers': 25}
}

# Create DataFrame
pd.DataFrame.from_dict(d)

Store 1                                     Store 2  \
                           2020                  2021                  2020   
Sales      {'A': 100, 'B': 200}  {'A': 150, 'B': 200}  {'A': 200, 'B': 200}   
Customers                    10                    15                    20   

                                 
                           2021  
Sales      {'A': 250, 'B': 200}  
Customers                    25

In [14]:
f = {
    ('Store 1', '2020'): {'Sales': 100, 'Customers': 10},
    ('Store 1', '2021'): {'Sales': 150, 'Customers': 15},
    ('Store 2', '2020'): {'Sales': 200, 'Customers': 20},
    ('Store 2', '2021'): {'Sales': 250, 'Customers': 25}
}

# Creating MultiIndex from the dictionary keys
index = pd.MultiIndex.from_tuples(f.keys(), names=['Store', 'Year'])

# Creating the DataFrame
df = pd.DataFrame([values for values in f.values()], index=index)
df

Sales  Customers
Store   Year                  
Store 1 2020    100         10
        2021    150         15
Store 2 2020    200         20
        2021    250         25

In [18]:
d = {
    ('Sales', 'Total'): [100, 150, 200, 250],
    ('Sales', 'Online'): [50, 70, 100, 150],
    ('Customers', 'Total'): [20, 25, 30, 35],
    ('Customers', 'Online'): [10, 15, 20, 25]
}

# Step 2: Create a MultiIndex for the rows
row_index = pd.MultiIndex.from_tuples([
    ('Store 1', '2020'),
    ('Store 1', '2021'),
    ('Store 2', '2020'),
    ('Store 2', '2021')
], names=['Store', 'Year'])

# Step 3: Create a MultiIndex for the columns
column_index = pd.MultiIndex.from_tuples([
    ('Sales', 'Total'),
    ('Sales', 'Online'),
    ('Customers', 'Total'),
    ('Customers', 'Online')
], names=['Metric', 'Type'])

# Step 4: Create the DataFrame
df = pd.DataFrame(np.arange(16).reshape(4, 4), index=row_index, columns=column_index)
df

Metric       Sales        Customers       
Type         Total Online     Total Online
Store   Year                              
Store 1 2020     0      1         2      3
        2021     4      5         6      7
Store 2 2020     8      9        10     11
        2021    12     13        14     15

In [15]:
[values for values in f.values()]

[{'Sales': 100, 'Customers': 10},
 {'Sales': 150, 'Customers': 15},
 {'Sales': 200, 'Customers': 20},
 {'Sales': 250, 'Customers': 25}]